<a href="https://colab.research.google.com/github/CakeNuthep/Create_Knowledge_Graph/blob/main/royin_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import pandas as pd

# Convert String to JSON

In [72]:
path_file = '/content/drive/MyDrive/KnowledgeGraph/royin.json'

In [73]:
f = open(path_file,"r",encoding='utf8')

In [74]:
f = open(path_file, "r",encoding="utf8")
count = 0
content = []
string_json = ''
begin = '['
end = ']'
split = ','

string_json += begin
while(True):
    # Read a line.
    line = f.readline()
    # When readline returns an empty string, the file is fully read.
    if line == "":
        print("::DONE::")
        break
    if count != 0:
      string_json += split
    string_json += line
    content.append(line)
    count +=1

string_json += end

::DONE::


In [75]:
import json
data = json.loads(string_json)

In [76]:
data

[{'_id': '123',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'ธรรมศาสตร์และการเมือง',
   'name': 'ไพฑูรย์ พงศะบุตร (ราชบัณฑิต)',
   'subject': 'สังคมศาสตร์',
   'title': 'ศาสตราจารย์กิตติคุณ'},
  '_type': 'royin'},
 {'_id': '120',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'วิทยาศาสตร์',
   'name': 'ไพรัช ธัชยพงษ์ (ภาคีสมาชิก)',
   'subject': 'วิทยาศาสตร์ประยุกต์',
   'title': 'ศาสตราจารย์ ดร.'},
  '_type': 'royin'},
 {'_id': '73',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'วิทยาศาสตร์',
   'name': 'วิสุทธิ์ ใบไม้ (ราชบัณฑิต)',
   'subject': 'วิทยาศาสตร์ชีวภาพ',
   'title': 'ศาสตราจารย์เกียรติคุณ ดร.'},
  '_type': 'royin'},
 {'_id': '92',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department': 'วิทยาศาสตร์',
   'name': 'ระวี ภาวิไล (ราชบัณฑิต)',
   'subject': 'วิทยาศาสตร์กายภาพ',
   'title': 'ศาสตราจารย์กิตติคุณ ดร.'},
  '_type': 'royin'},
 {'_id': '60',
  '_index': 'royin',
  '_score': 1,
  '_source': {'department':

In [77]:
df_publish = pd.json_normalize(data)

In [78]:
df_publish

,_index,_type,_id,_score,_source.subject,_source.name,_source.title,_source.department
0,royin,royin,123,1,สังคมศาสตร์,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),ศาสตราจารย์กิตติคุณ,ธรรมศาสตร์และการเมือง
1,royin,royin,120,1,วิทยาศาสตร์ประยุกต์,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,วิทยาศาสตร์
2,royin,royin,73,1,วิทยาศาสตร์ชีวภาพ,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),ศาสตราจารย์เกียรติคุณ ดร.,วิทยาศาสตร์
3,royin,royin,92,1,วิทยาศาสตร์กายภาพ,ระวี ภาวิไล (ราชบัณฑิต),ศาสตราจารย์กิตติคุณ ดร.,วิทยาศาสตร์
4,royin,royin,60,1,สังคมศาสตร์,สนั่น ปัทมะทิน,ศาสตราจารย์,ธรรมศาสตร์และการเมือง
...,...,...,...,...,...,...,...,...
214,royin,royin,53,1,ปรัชญา,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,ธรรมศาสตร์และการเมือง
215,royin,royin,68,1,วิจิตรศิลป์,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,ศิลปกรรม
216,royin,royin,17,1,ปรัชญา,วรรณไวทยากร วรวรรณ,พลตรี พระเจ้าวรวงศ์เธอ กรมหมื่นนราธิปพงศ์ประพั...,ธรรมศาสตร์และการเมือง
217,royin,royin,31,1,วิทยาศาสตร์ชีวภาพ,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),ศาสตราจารย์ ดร.,วิทยาศาสตร์


In [79]:
df_new = pd.concat([df_publish])

# Create Author Table

In [80]:
uniq_name = df_new['_source.name'].unique()

In [81]:
uniq_name

array(['ไพฑูรย์ พงศะบุตร (ราชบัณฑิต)', 'ไพรัช ธัชยพงษ์ (ภาคีสมาชิก)',
       'วิสุทธิ์ ใบไม้ (ราชบัณฑิต)', 'ระวี ภาวิไล (ราชบัณฑิต)',
       'สนั่น ปัทมะทิน', 'สมชาติ โสภณรณฤทธิ์ (ราชบัณฑิต)',
       'โสภา ชูพิกุลชัย ชปีลมันน์ (ราชบัณฑิต)',
       'สิวลี ศิริไล (ภาคีสมาชิก)', 'สมใจ นิ่มเล็ก',
       'มนุวดี หังสพฤกษ์ (ราชบัณฑิต)', 'พรสรรค์ วัฒนางกูร (ภาคีสมาชิก)',
       'ยงค์วิมล เลณบุรี (ราชบัณฑิต)', 'เสฐียร พันธรังษี',
       'สวัสดิ์ ตันติสุข', 'ประเวศ วะสี (ราชบัณฑิตกิตติมศักดิ์)',
       'วรศักดิ์ กนกนุกูลชัย (ภาคีสมาชิก)', 'ประเสริฐ ณ นคร (ราชบัณฑิต)',
       'พูนพิศ อมาตยกุล (ราชบัณฑิตกิตติมศักดิ์)',
       'พูนทรัพย์ นพวงศ์ ณ อยุธยา (ราชบัณฑิตกิตติมศักดิ์)',
       'ไขแสง ศุขะวัฒนะ (ราชบัณฑิต)', 'ปกรณ์ อดุลพันธุ์ (ราชบัณฑิต)',
       'ผลิน ภู่จรูญ (ราชบัณฑิต)', 'สมชัย บวรกิตติ (ราชบัณฑิต)',
       'ปณต มิคะเสน (ราชบัณฑิต)', 'ลิขิต ธีรเวคิน (ราชบัณฑิต)',
       'ยง ภู่วรวรรณ (ภาคีสมาชิก)', 'วิชิตวงศ์ ณ ป้อมเพชร (ราชบัณฑิต)',
       'กุหลาบ มัลลิกะมาส', 'มีชัย ฤชุพันธุ์ (ราชบัณฑ

In [82]:
df_unique_name = pd.DataFrame(uniq_name,columns=['_source.name'])

In [83]:
df_unique_name

,_source.name
0,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต)
1,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก)
2,วิสุทธิ์ ใบไม้ (ราชบัณฑิต)
3,ระวี ภาวิไล (ราชบัณฑิต)
4,สนั่น ปัทมะทิน
...,...
214,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก)
215,วิรุณ ตั้งเจริญ (ภาคีสมาชิก)
216,วรรณไวทยากร วรวรรณ
217,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก)


In [84]:
start_id = 548621
df_unique_name['id'] = df_unique_name.index + start_id

In [85]:
df_unique_name

,_source.name,id
0,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),548621
1,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),548622
2,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),548623
3,ระวี ภาวิไล (ราชบัณฑิต),548624
4,สนั่น ปัทมะทิน,548625
...,...,...
214,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),548835
215,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),548836
216,วรรณไวทยากร วรวรรณ,548837
217,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),548838


# df_author

In [86]:
def is_english(str):
    for i in str:
        if i in ('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'):
            return True
    else:    
        return False

def is_thai(str):
    for i in str:
        if i in ('ก','ข','ฃ','ค','ฅ','ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ'):
            return True
    else:
        return False

def language(text):
    if is_english(text):
      return 'eng'
    elif is_thai(text):
      return 'th'
    else:
      return 'other'

def strip_row(text):
    text = text.strip()
    text = ' '.join(text.split())
    return text

def first_name(list_text):
    name = ''
    if len(list_text) > 0:
      name = strip_row(list_text[0])
    return name

def check_first_author(row):
    if pd.isna(row['first_name']):
      return 'no'
    elif pd.isna(row['new_name']):
      return 'no'
    elif row['first_name'] == '':
      return 'no'
    elif row['first_name'] == row['new_name']:
      return 'yes'
    else:
      return 'no'

In [87]:
df_unique_name = df_unique_name.rename(columns={"_source.name": "authFullName"})

In [88]:
df_unique_name['language'] = df_unique_name['authFullName'].apply(language)

In [89]:
result_author_thai = df_unique_name[df_unique_name['language']=='th']

result_author_eng = df_unique_name[df_unique_name['language']=='eng']

result_author_other = df_unique_name[df_unique_name['language']=='other']

In [90]:
result_author_thai

,authFullName,id,language
0,ไพฑูรย์ พงศะบุตร (ราชบัณฑิต),548621,th
1,ไพรัช ธัชยพงษ์ (ภาคีสมาชิก),548622,th
2,วิสุทธิ์ ใบไม้ (ราชบัณฑิต),548623,th
3,ระวี ภาวิไล (ราชบัณฑิต),548624,th
4,สนั่น ปัทมะทิน,548625,th
...,...,...,...
214,สิทธิ์ บุตรอินทร์ (ภาคีสมาชิก),548835,th
215,วิรุณ ตั้งเจริญ (ภาคีสมาชิก),548836,th
216,วรรณไวทยากร วรวรรณ,548837,th
217,สิริวัฒน์ วงษ์ศิริ (ภาคีสมาชิก),548838,th


In [91]:
result_author_eng

,authFullName,id,language


In [92]:
result_author_other

,authFullName,id,language


# Create Auth Detail

In [93]:
result_all = pd.merge(df_new,
                       df_unique_name,
                       left_on='_source.name',
                       right_on='authFullName',
                       how='left')

In [94]:
def fulltitle(str):
    c = ['ว่าที่','จอมพลพิเศษหญิง','พลเอกพิเศษหญิง','พลโทพิเศษหญิง','พลตรีพิเศษหญิง','พลจัตวาพิเศษหญิง','พันเอกพิเศษหญิง','พันโทพิเศษหญิง','พันตรีพิเศษหญิง','ร้อยเอกพิเศษหญิง','ร้อยโทพิเศษหญิง','ร้อยตรีพิเศษหญิง','จ่าสิบเอกพิเศษพิเศษหญิง','จ่าสิบเอกพิเศษหญิง','จ่าสิบโทพิเศษหญิง','จ่าสิบตรีพิเศษหญิง','สิบเอกพิเศษหญิง','สิบโทพิเศษหญิง','สิบตรีพิเศษหญิง','จอมพลเรือพิเศษหญิง','พลเรือเอกพิเศษหญิง','พลเรือโทพิเศษหญิง','พลเรือตรีพิเศษหญิง','พลเรือจัตวาพิเศษหญิง','นาวาเอกพิเศษหญิง','นาวาโทพิเศษหญิง','นาวาตรีพิเศษหญิง','เรือเอกพิเศษหญิง','เรือโทพิเศษหญิง','เรือตรีพิเศษหญิง','พันจ่าเอกพิเศษหญิง','พันจ่าโทพิเศษหญิง','พันจ่าตรีพิเศษหญิง','จ่าเอกพิเศษหญิง','จ่าโทพิเศษหญิง','จ่าตรีพิเศษหญิง','จอมพลอากาศพิเศษหญิง','พลอากาศเอกพิเศษหญิง','พลอากาศโทพิเศษหญิง','พลอากาศตรีพิเศษหญิง','พลอากาศจัตวาพิเศษหญิง','นาวาอากาศเอกพิเศษหญิง','นาวาอากาศโทพิเศษหญิง','นาวาอากาศตรีพิเศษหญิง','เรืออากาศเอกพิเศษหญิง','เรืออากาศโทพิเศษหญิง','เรืออากาศตรีพิเศษหญิง','พันจ่าอากาศเอกพิเศษหญิง','พันจ่าอากาศโทพิเศษหญิง','พันจ่าอากาศตรีพิเศษหญิง','จ่าอากาศเอกพิเศษหญิง','จ่าอากาศโทพิเศษหญิง','จ่าอากาศตรีพิเศษหญิง','พลตำรวจเอกพิเศษหญิง','พลตำรวจโทพิเศษหญิง','พลตำรวจตรีพิเศษหญิง','พลตำรวจจัตวาพิเศษหญิง','พันตำรวจเอกพิเศษหญิง','พันตำรวจโทพิเศษหญิง','พันตำรวจตรีพิเศษหญิง','ร้อยตำรวจเอกพิเศษหญิง','ร้อยตำรวจโทพิเศษหญิง','ร้อยตำรวจตรีพิเศษหญิง','ดาบตำรวจพิเศษหญิง','จ่าสิบตำรวจพิเศษหญิง','สิบตำรวจเอกพิเศษหญิง','สิบตำรวจโทพิเศษหญิง','สิบตำรวจตรีพิเศษหญิง','จอมพลพิเศษ','พลเอกพิเศษ','พลโทพิเศษ','พลตรีพิเศษ','พลจัตวาพิเศษ','พันเอกพิเศษ','พันโทพิเศษ','พันตรีพิเศษ','ร้อยเอกพิเศษ','ร้อยโทพิเศษ','ร้อยตรีพิเศษ','จ่าสิบเอกพิเศษพิเศษ','จ่าสิบเอกพิเศษ','จ่าสิบโทพิเศษ','จ่าสิบตรีพิเศษ','สิบเอกพิเศษ','สิบโทพิเศษ','สิบตรีพิเศษ','จอมพลเรือพิเศษ','พลเรือเอกพิเศษ','พลเรือโทพิเศษ','พลเรือตรีพิเศษ','พลเรือจัตวาพิเศษ','นาวาเอกพิเศษ','นาวาโทพิเศษ','นาวาตรีพิเศษ','เรือเอกพิเศษ','เรือโทพิเศษ','เรือตรีพิเศษ','พันจ่าเอกพิเศษ','พันจ่าโทพิเศษ','พันจ่าตรีพิเศษ','จ่าเอกพิเศษ','จ่าโทพิเศษ','จ่าตรีพิเศษ','จอมพลอากาศพิเศษ','พลอากาศเอกพิเศษ','พลอากาศโทพิเศษ','พลอากาศตรีพิเศษ','พลอากาศจัตวาพิเศษ','นาวาอากาศเอกพิเศษ','นาวาอากาศโทพิเศษ','นาวาอากาศตรีพิเศษ','เรืออากาศเอกพิเศษ','เรืออากาศโทพิเศษ','เรืออากาศตรีพิเศษ','พันจ่าอากาศเอกพิเศษ','พันจ่าอากาศโทพิเศษ','พันจ่าอากาศตรีพิเศษ','จ่าอากาศเอกพิเศษ','จ่าอากาศโทพิเศษ','จ่าอากาศตรีพิเศษ','พลตำรวจเอกพิเศษ','พลตำรวจโทพิเศษ','พลตำรวจตรีพิเศษ','พลตำรวจจัตวาพิเศษ','พันตำรวจเอกพิเศษ','พันตำรวจโทพิเศษ','พันตำรวจตรีพิเศษ','ร้อยตำรวจเอกพิเศษ','ร้อยตำรวจโทพิเศษ','ร้อยตำรวจตรีพิเศษ','ดาบตำรวจพิเศษ','จ่าสิบตำรวจพิเศษ','สิบตำรวจเอกพิเศษ','สิบตำรวจโทพิเศษ','สิบตำรวจตรีพิเศษ','จอมพลหญิง','พลเอกหญิง','พลโทหญิง','พลตรีหญิง','พลจัตวาหญิง','พันเอกหญิง','พันโทหญิง','พันตรีหญิง','ร้อยเอกหญิง','ร้อยโทหญิง','ร้อยตรีหญิง','จ่าสิบเอกพิเศษหญิง','จ่าสิบเอกหญิง','จ่าสิบโทหญิง','จ่าสิบตรีหญิง','สิบเอกหญิง','สิบโทหญิง','สิบตรีหญิง','จอมพลเรือหญิง','พลเรือเอกหญิง','พลเรือโทหญิง','พลเรือตรีหญิง','พลเรือจัตวาหญิง','นาวาเอกหญิง','นาวาโทหญิง','นาวาตรีหญิง','เรือเอกหญิง','เรือโทหญิง','เรือตรีหญิง','พันจ่าเอกหญิง','พันจ่าโทหญิง','พันจ่าตรีหญิง','จ่าเอกหญิง','จ่าโทหญิง','จ่าตรีหญิง','จอมพลอากาศหญิง','พลอากาศเอกหญิง','พลอากาศโทหญิง','พลอากาศตรีหญิง','พลอากาศจัตวาหญิง','นาวาอากาศเอกหญิง','นาวาอากาศโทหญิง','นาวาอากาศตรีหญิง','เรืออากาศเอกหญิง','เรืออากาศโทหญิง','เรืออากาศตรีหญิง','พันจ่าอากาศเอกหญิง','พันจ่าอากาศโทหญิง','พันจ่าอากาศตรีหญิง','จ่าอากาศเอกหญิง','จ่าอากาศโทหญิง','จ่าอากาศตรีหญิง','พลตำรวจเอกหญิง','พลตำรวจโทหญิง','พลตำรวจตรีหญิง','พลตำรวจจัตวาหญิง','พันตำรวจเอกหญิง','พันตำรวจโทหญิง','พันตำรวจตรีหญิง','ร้อยตำรวจเอกหญิง','ร้อยตำรวจโทหญิง','ร้อยตำรวจตรีหญิง','ดาบตำรวจหญิง','จ่าสิบตำรวจหญิง','สิบตำรวจเอกหญิง','สิบตำรวจโทหญิง','สิบตำรวจตรีหญิง','จอมพล','พลเอก','พลโท','พลตรี','พลจัตวา','พันเอก','พันโท','พันตรี','ร้อยเอก','ร้อยโท','ร้อยตรี','จ่าสิบเอกพิเศษ','จ่าสิบเอก','จ่าสิบโท','จ่าสิบตรี','สิบเอก','สิบโท','สิบตรี','จอมพลเรือ','พลเรือเอก','พลเรือโท','พลเรือตรี','พลเรือจัตวา','นาวาเอก','นาวาโท','นาวาตรี','เรือเอก','เรือโท','เรือตรี','พันจ่าเอก','พันจ่าโท','พันจ่าตรี','จ่าเอก','จ่าโท','จ่าตรี','จอมพลอากาศ','พลอากาศเอก','พลอากาศโท','พลอากาศตรี','พลอากาศจัตวา','นาวาอากาศเอก','นาวาอากาศโท','นาวาอากาศตรี','เรืออากาศเอก','เรืออากาศโท','เรืออากาศตรี','พันจ่าอากาศเอก','พันจ่าอากาศโท','พันจ่าอากาศตรี','จ่าอากาศเอก','จ่าอากาศโท','จ่าอากาศตรี','พลตำรวจเอก','พลตำรวจโท','พลตำรวจตรี','พลตำรวจจัตวา','พันตำรวจเอก','พันตำรวจโท','พันตำรวจตรี','ร้อยตำรวจเอก','ร้อยตำรวจโท','ร้อยตำรวจตรี','ดาบตำรวจ','จ่าสิบตำรวจ','สิบตำรวจเอก','สิบตำรวจโท','สิบตำรวจตรี','หม่อมหลวง','ผู้ช่วยศาสตราจารย์พิเศษ','รองศาสตราจารย์พิเศษ','ศาสตราจารย์พิเศษ','อาจารย์พิเศษ','ผู้ช่วยศาสตราจารย์','รองศาสตราจารย์','ศาสตราจารย์กิตติคุณ','ศาสตราจารย์เกียรติยศ','ศาสตราจารย์','อาจารย์','ทันตแพทย์หญิง','ทันตแพทย์','แพทย์หญิง','นายแพทย์','นางสาว','นาง','นาย','คุณหญิง','ท่านหญิง','ท่านผู้หญิง','หม่อมราชวงศ์หญิง','หม่อมราชวงศ์','หม่อมเจ้าหญิง','หม่อมเจ้า','ฯพณฯ']
    d = ['ผู้ช่วยศาสตรจารย์','รองศาตราจารย์','รมว.กระทรวงพาณิชย์']
    for i in c:
        if str.endswith(i) == True or str in d:
            return True
    else:
        return False
def shorttitle(str):
    c  = ['พล.อ.','พล.ท.','พล.ต','พล.จ.','พ.อ.','พ.ท.','พ.ต.','ร.อ.','ร.ท.','ร.ต.','จ.ส.อ.','จ.ส.ท.','จ.ส.ต.','ส.อ.','ส.ท.','ส.ต.','พล.ร.อ.','พล.ร.ท.','พล.ร.ต.','พล.ร.จ.','น.อ.','น.ท.','น.ต.','พ.จ.อ.','พ.จ.ท.','พ.จ.ต.','จ.อ.','จ.ท.','จ.ต.','พล.อ.อ.','พล.อ.ท.','พล.อ.ต.','พล.อ.จ.','พ.อ.อ.','พ.อ.ท.','พ.อ.ต.','พล.ต.อ.','พล.ต.ท.','พล.ต.ต.','พล.ต.จ.','พ.ต.อ.','พ.ต.ท.','พ.ต.ต.','ร.ต.อ.','ร.ต.ท.','ร.ต.ต.','ด.ต.','จ.ส.ต.','ส.ต.อ.','ส.ต.ท.','ส.ต.ต.','พญ.','นพ.','ดร.','น.ส.','นส.','ม.ล.','ม.ร.ว.','ม.จ.','ผศ.','รศ.','ศ.','อ.','น.สพ.','สพ.ญ.','ภญ.','ภกญ.']
    d = ['พญ','นพ','ดร','ผศ','รศ','ศ','อ','รต.']
    for i in c:
        if str.endswith(i) == True or str in d or str.endswith('.หญิง') == True or str.endswith('.(หญิง)') == True or str.endswith('.(พิเศษ)') == True or str.endswith('.พิเศษ') == True or str.endswith('ฯพณฯ') == True :
            return True
    else:
        return False       

def eng_tt_frontname(str):
    c = ('Ph.D.','Associate.','Assoc.','Asso.','Asst.','Assist.','Professor.','Prof.','Dr.','Mr.','Ms.','associate.','assoc.','asso.','asst.','assist.','professor.','prof.','dr.','mr.','ms.','ASSOCIATE.','ASSOC.','ASSO.','ASST.','ASSIST.','PROFESSOR.','PROF.','DR.','MR.','MS.')
    for i in c:
        if str.find(i)>-1:
            return True
    else:
        return False
def engtitle(str):
    c = ('Md.','MD.','md.','Mrs.','MRS.','mrs.','Ph.D.','Associate.','Assoc.','Asso.','Asst.','Assist.','Professor.','Prof.','Dr.','Mr.','Ms.','associate.','assoc.','asso.','asst.','assist.','professor.','prof.','dr.','mr.','ms.','ASSOCIATE.','ASSOC.','ASSO.','ASST.','ASSIST.','PROFESSOR.','PROF.','DR.','MR.','MS.')
    d = ('Md','MD','md','Mrs','MRS','mrs','Ph.D','PhD','miss','Miss','Associate','Assoc','Asso','Asst','Assist','Professor','Prof','Dr','Mr','Ms','associate','assoc','asso','asst','assist','professor','prof','dr','mr','ms','ASSOCIATE','ASSOC','ASSO','ASST','ASSIST','PROFESSOR','PROF','DR','MR','MS')
    for i in c:
        if str.endswith(i) == True or str in d:
             return True
    else:
        return False

In [95]:
result_all.columns

Index(['_index', '_type', '_id', '_score', '_source.subject', '_source.name',
       '_source.title', '_source.department', 'authFullName', 'id',
       'language'],
      dtype='object')

In [96]:
auth_detail = result_all

In [97]:
auth_detail = auth_detail.rename(columns={"id": "auth_ID","language":"author_lang","_source.title":"titleName", "_source.subject":"subject","_source.department":"department"})

In [98]:
auth_detail.columns

Index(['_index', '_type', '_id', '_score', 'subject', '_source.name',
       'titleName', 'department', 'authFullName', 'auth_ID', 'author_lang'],
      dtype='object')

In [99]:
auth_detail = auth_detail.drop(['_index', '_type','_id', '_score',
       '_source.name','authFullName'],axis=1)

In [100]:
## auth_detail['id'] = ''
# auth_detail['auth_ID'] = ''
# auth_detail['author_lang'] = ''
# auth_detail['titleName'] = ''
auth_detail['tel'] = ''
auth_detail['email'] = ''
# auth_detail['subject'] = ''
# auth_detail['department'] = ''

In [101]:
all_column =  ["auth_ID","author_lang","titleName","tel","email","subject","department"]
len(all_column)

7

In [102]:
len(auth_detail.columns)

7

In [103]:
auth_detail.columns

Index(['subject', 'titleName', 'department', 'auth_ID', 'author_lang', 'tel',
       'email'],
      dtype='object')

In [104]:
auth_detail = auth_detail[all_column]

In [105]:
auth_detail

,auth_ID,author_lang,titleName,tel,email,subject,department
0,548621,th,ศาสตราจารย์กิตติคุณ,,,สังคมศาสตร์,ธรรมศาสตร์และการเมือง
1,548622,th,ศาสตราจารย์ ดร.,,,วิทยาศาสตร์ประยุกต์,วิทยาศาสตร์
2,548623,th,ศาสตราจารย์เกียรติคุณ ดร.,,,วิทยาศาสตร์ชีวภาพ,วิทยาศาสตร์
3,548624,th,ศาสตราจารย์กิตติคุณ ดร.,,,วิทยาศาสตร์กายภาพ,วิทยาศาสตร์
4,548625,th,ศาสตราจารย์,,,สังคมศาสตร์,ธรรมศาสตร์และการเมือง
...,...,...,...,...,...,...,...
214,548835,th,ศาสตราจารย์ ดร.,,,ปรัชญา,ธรรมศาสตร์และการเมือง
215,548836,th,ศาสตราจารย์ ดร.,,,วิจิตรศิลป์,ศิลปกรรม
216,548837,th,พลตรี พระเจ้าวรวงศ์เธอ กรมหมื่นนราธิปพงศ์ประพั...,,,ปรัชญา,ธรรมศาสตร์และการเมือง
217,548838,th,ศาสตราจารย์ ดร.,,,วิทยาศาสตร์ชีวภาพ,วิทยาศาสตร์


# Author Expert

In [106]:
auth_expert = result_all

In [107]:
auth_expert.columns

Index(['_index', '_type', '_id', '_score', '_source.subject', '_source.name',
       '_source.title', '_source.department', 'authFullName', 'id',
       'language'],
      dtype='object')

In [108]:
auth_expert = auth_expert.rename(columns={"id": "auth_ID","language":"author_lang", "_source.subject":"subject" })

In [109]:
auth_expert.columns

Index(['_index', '_type', '_id', '_score', 'subject', '_source.name',
       '_source.title', '_source.department', 'authFullName', 'auth_ID',
       'author_lang'],
      dtype='object')

In [110]:
auth_expert = auth_expert.drop(['_index', '_type','_id', '_score',
       '_source.name','_source.title','_source.department','authFullName'],axis=1)

In [111]:
## auth_expert['id'] = ''
# auth_expert['auth_ID'] = ''
# auth_expert['author_lang'] = ''
# auth_expert['subject'] = ''
auth_expert['expertise'] = ''

In [112]:
all_column =  ["auth_ID","author_lang","subject","expertise"]
len(all_column)

4

In [113]:
len(auth_expert.columns)

4

In [114]:
auth_expert.columns

Index(['subject', 'auth_ID', 'author_lang', 'expertise'], dtype='object')

In [115]:
auth_expert = auth_expert[all_column]

In [116]:
auth_expert

,auth_ID,author_lang,subject,expertise
0,548621,th,สังคมศาสตร์,
1,548622,th,วิทยาศาสตร์ประยุกต์,
2,548623,th,วิทยาศาสตร์ชีวภาพ,
3,548624,th,วิทยาศาสตร์กายภาพ,
4,548625,th,สังคมศาสตร์,
...,...,...,...,...
214,548835,th,ปรัชญา,
215,548836,th,วิจิตรศิลป์,
216,548837,th,ปรัชญา,
217,548838,th,วิทยาศาสตร์ชีวภาพ,


# Export

In [117]:
# author

result_author_thai[['id','authFullName']].to_csv('royin_author_thai.csv')
result_author_eng[['id','authFullName']].to_csv('royin_author_eng.csv')
result_author_other[['id','authFullName']].to_csv('royin_author_other.csv')

In [118]:
# author detail , education, expert
auth_detail.index.name = 'id'
auth_expert.index.name = 'id'

auth_detail.to_csv('royin_author_detail.csv')
auth_expert.to_csv('royin_author_expert.csv')